In [30]:
import requests
import pandas as pd
from sqlalchemy import create_engine, inspect
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
import csv
import numpy as np



In [2]:
#reads in the csv
csv_file = "../data/summer.csv"
directory_db = pd.read_csv(csv_file)

directory_db

,Year,City,Sport,Discipline,Athlete,Country,Gender,Event,Medal
0,1896,Athens,Aquatics,Swimming,"HAJOS, Alfred",HUN,Men,100M Freestyle,Gold
1,1896,Athens,Aquatics,Swimming,"HERSCHMANN, Otto",AUT,Men,100M Freestyle,Silver
2,1896,Athens,Aquatics,Swimming,"DRIVAS, Dimitrios",GRE,Men,100M Freestyle For Sailors,Bronze
3,1896,Athens,Aquatics,Swimming,"MALOKINIS, Ioannis",GRE,Men,100M Freestyle For Sailors,Gold
4,1896,Athens,Aquatics,Swimming,"CHASAPIS, Spiridon",GRE,Men,100M Freestyle For Sailors,Silver
...,...,...,...,...,...,...,...,...,...
31160,2012,London,Wrestling,Wrestling Freestyle,"JANIKOWSKI, Damian",POL,Men,Wg 84 KG,Bronze
31161,2012,London,Wrestling,Wrestling Freestyle,"REZAEI, Ghasem Gholamreza",IRI,Men,Wg 96 KG,Gold
31162,2012,London,Wrestling,Wrestling Freestyle,"TOTROV, Rustam",RUS,Men,Wg 96 KG,Silver
31163,2012,London,Wrestling,Wrestling Freestyle,"ALEKSANYAN, Artur",ARM,Men,Wg 96 KG,Bronze


In [40]:
#groups country by Country and Year
country_medal_db=directory_db.groupby(['Country','Year','City']).count()
country_medal_db=country_medal_db[['Medal']]
country_medal_db.reset_index(inplace=True)

#rearranges by year
country_medal_db.sort_values('Year', ascending=True,inplace=True)
country_medal_db.set_index(['Year','Country'],inplace=True)


country_medal_db

City  Medal
Year Country               
1896 AUT      Athens      5
     GBR      Athens      7
     GER      Athens     33
     DEN      Athens      6
     GRE      Athens     52
...              ...    ...
2012 GAB      London      1
     FRA      London     82
     RUS      London    130
     GER      London     94
     QAT      London      2

[1157 rows x 2 columns]

In [41]:
#calculates Medal accumalitve sum - though I didn't end up using it
country_medal_db['Medal_Sum'] = country_medal_db.groupby('Country').cumsum()
country_medal_db

City  Medal  Medal_Sum
Year Country                          
1896 AUT      Athens      5          5
     GBR      Athens      7          7
     GER      Athens     33         33
     DEN      Athens      6          6
     GRE      Athens     52         52
...              ...    ...        ...
2012 GAB      London      1          1
     FRA      London     82       1396
     RUS      London    130        768
     GER      London     94       1305
     QAT      London      2          4

[1157 rows x 3 columns]

In [42]:
#exports csv
country_medal_db.to_csv('../Data/summer_country_medals.csv',index=True)

In [35]:
#groups country by Country and Year
olympic_summer_db=directory_db.groupby(['City','Year']).count()

#country_medal_db=country_medal_db[['Medal']]
olympic_summer_db.reset_index(inplace=True)
olympic_summer_db=olympic_summer_db[['City','Year']]
#rearranges by year
olympic_summer_db.sort_values('Year', ascending=True,inplace=True)
olympic_summer_db.set_index(['Year'],inplace=True)


olympic_summer_db.to_csv('../Data/summer_game_cities.csv',index=True)
olympic_summer_db

,City
Year,
1896,Athens
1900,Paris
1904,St Louis
1908,London
1912,Stockholm
1920,Antwerp
1924,Paris
1928,Amsterdam
1932,Los Angeles


------------------------------Start SQL File Here-----------------------------

In [2]:
database_path= "../data/Olympics.sqlite"

In [15]:
engine = create_engine(f"sqlite:///{database_path}")
conn = engine.connect()

In [16]:
Base = automap_base()

In [17]:
Base.prepare(engine, reflect=True)

In [31]:
inspector = inspect(conn)

In [34]:
inspector.get_table_names()

['countryData', 'olympicsData']

In [50]:
data=pd.read_sql("SELECT * FROM olympicsData",conn)
data.head(10)

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,15,Arvo Ossian Aaltonen,M,30,NA,NA,Finland,FIN,1920 Summer,1920,Summer,Antwerpen,Swimming,Swimming Men's 200 metres Breaststroke,Bronze
1,15,Arvo Ossian Aaltonen,M,30,NA,NA,Finland,FIN,1920 Summer,1920,Summer,Antwerpen,Swimming,Swimming Men's 400 metres Breaststroke,Bronze
2,16,Juhamatti Tapio Aaltonen,M,28,184,85,Finland,FIN,2014 Winter,2014,Winter,Sochi,Ice Hockey,Ice Hockey Men's Ice Hockey,Bronze
3,17,Paavo Johannes Aaltonen,M,28,175,64,Finland,FIN,1948 Summer,1948,Summer,London,Gymnastics,Gymnastics Men's Individual All-Around,Bronze
4,17,Paavo Johannes Aaltonen,M,32,175,64,Finland,FIN,1952 Summer,1952,Summer,Helsinki,Gymnastics,Gymnastics Men's Team All-Around,Bronze
5,20,Kjetil Andr Aamodt,M,20,176,85,Norway,NOR,1992 Winter,1992,Winter,Albertville,Alpine Skiing,Alpine Skiing Men's Giant Slalom,Bronze
6,20,Kjetil Andr Aamodt,M,22,176,85,Norway,NOR,1994 Winter,1994,Winter,Lillehammer,Alpine Skiing,Alpine Skiing Men's Super G,Bronze
7,29,Willemien Aardenburg,F,22,NA,NA,Netherlands,NED,1988 Summer,1988,Summer,Seoul,Hockey,Hockey Women's Hockey,Bronze
8,37,Ann Kristin Aarnes,F,23,182,64,Norway,NOR,1996 Summer,1996,Summer,Atlanta,Football,Football Women's Football,Bronze
9,40,Roald Edgar Aas,M,23,NA,NA,Norway,NOR,1952 Winter,1952,Winter,Oslo,Speed Skating,"Speed Skating Men's 1,500 metres",Bronze


In [52]:
data.groupby(["Team","Year","City"]).count()

ID  Name  Sex  Age  Height  Weight  \
Team                  Year City                                              
A North American Team 1900 Paris         4     4    4    4       4       4   
Afghanistan           2008 Beijing       1     1    1    1       1       1   
                      2012 London        1     1    1    1       1       1   
Algeria               1984 Los Angeles   2     2    2    2       2       2   
                      1992 Barcelona     2     2    2    2       2       2   
...                                     ..   ...  ...  ...     ...     ...   
Zambia                1996 Atlanta       1     1    1    1       1       1   
Zimbabwe              1980 Moskva       15    15   15   15      15      15   
                      2004 Athina        3     3    3    3       3       3   
                      2008 Beijing       4     4    4    4       4       4   
Zut                   1908 London        3     3    3    3       3       3   

                                        NOC  Games  Season  Sport  Event  \
Team                  Year City                                            
A North American Team 1900 Paris          4      4       4      4      4   
Afghanistan           2008 Beijing        1      1       1      1      1   
                      2012 London         1      1       1      1      1   
Algeria               1984 Los Angeles    2      2       2      2      2   
                      1992 Barcelona      2      2       2      2      2   
...                                     ...    ...     ...    ...    ...   
Zambia                1996 Atlanta        1      1       1      1      1   
Zimbabwe              1980 Moskva        15     15      15     15     15   
                      2004 Athina         3      3       3      3      3   
                      2008 Beijing        4      4       4      4      4   
Zut                   1908 London         3      3       3      3      3   

                                        Medal  
Team                  Year City                
A North American Team 1900 Paris            4  
Afghanistan           2008 Beijing          1  
                      2012 London           1  
Algeria               1984 Los Angeles      2  
                      1992 Barcelona        2  
...                                       ...  
Zambia                1996 Atlanta          1  
Zimbabwe              1980 Moskva          15  
                      2004 Athina           3  
                      2008 Beijing          4  
Zut                   1908 London           3  

[2223 rows x 12 columns]